In [1]:
import numpy as np
import tifffile
from scipy.interpolate import RBFInterpolator
from scipy.ndimage import map_coordinates
import matplotlib.pyplot as plt

In [2]:
# --- 1. 画像読み込み ---
pathA = '/Volumes/BaffaloSSDPUTU3C1TB/rbak_data/scaled/rbak006_l.tif'
pathB = '/Volumes/BaffaloSSDPUTU3C1TB/rbak_data/scaled/rbak002_l.tif'
A = tifffile.imread(pathA)  # 画像A (shape: Z, Y, X)
B = tifffile.imread(pathB)  # 画像B (shape: Z, Y, X)

In [3]:
# --- 2. キーポイント設定 ---
# npyファイルからキーポイントを読み込む
points_A_file = "/Volumes/BaffaloSSDPUTU3C1TB/rbak_data/keypoints/rbak006_l.npy"
points_B_file = "/Volumes/BaffaloSSDPUTU3C1TB/rbak_data/keypoints/rbak002_l.npy"

points_A = np.load(points_A_file)  # .npyファイルからAのキーポイントを読み込み
points_B = np.load(points_B_file)  # .npyファイルからBのキーポイントを読み込み

if points_A.shape != points_B.shape or points_A.shape[1] != 3:
    raise ValueError("Keypoints in A and B must have the same shape and each point must have 3 coordinates (z, y, x).")

In [4]:
# --- 3. 変位ベクトル計算 ---
displacements = points_A - points_B  # A上でのBのズレ (shape: N, 3)

In [5]:
# --- 4. RBF補間で変形場を作成 ---
rbf = RBFInterpolator(points_A, displacements, smoothing=0.0, kernel='thin_plate_spline')

In [6]:
# ---- Step 5: Process Image in Chunks to Handle Large Data ----
Z, Y, X = A.shape
warped_B = np.zeros_like(A, dtype=B.dtype)  # Output image with the same shape as A

chunk_size = 10  # Number of Z slices to process in each chunk (adjust based on memory)

for z_start in range(0, Z, chunk_size):
    z_end = min(z_start + chunk_size, Z)

    # Generate coordinates for the current chunk
    zz, yy, xx = np.meshgrid(
        np.arange(z_start, z_end),  # Z range for this chunk
        np.arange(Y),               # Y range for the entire image
        np.arange(X),               # X range for the entire image
        indexing='ij'
    )
    coords_A_chunk = np.stack([zz.ravel(), yy.ravel(), xx.ravel()], axis=1)  # Shape: (chunk_size * Y * X, 3)

    # Compute displacement field for the current chunk
    displacement_field = rbf(coords_A_chunk)  # Shape: (chunk_size * Y * X, 3)

    # Calculate B's coordinates corresponding to A's coordinates
    coords_B_chunk = coords_A_chunk - displacement_field

    # Reshape coordinates to 3D for `map_coordinates`
    coords_B_z = coords_B_chunk[:, 0].reshape(z_end - z_start, Y, X)
    coords_B_y = coords_B_chunk[:, 1].reshape(z_end - z_start, Y, X)
    coords_B_x = coords_B_chunk[:, 2].reshape(z_end - z_start, Y, X)

    # Warp B using the calculated deformation field
    warped_B[z_start:z_end, :, :] = map_coordinates(
        B, [coords_B_z, coords_B_y, coords_B_x],
        order=1, mode='nearest'  # Linear interpolation, nearest boundary mode
    )

In [ ]:
# ---- Step 6: Save the Result ----
tifffile.imwrite("/Volumes/BaffaloSSDPUTU3C1TB/rbak_data/kptransformed/rbak002_l.tif", warped_B.astype('uint8'))
print("Warped image saved")

Warped image saved
